In [ ]:
! pip install pandas

In [28]:
# imports
import zipfile
import pandas as pd

In [29]:

# unzip

zip_file_path = "Import_Refusal_2014-present.zip"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

print("ZIP file has been successfully extracted.")


ZIP file has been successfully extracted.


## Loading and cleaning the dataset

In [30]:
try:
    df = pd.read_csv('REFUSAL_ENTRY_2014-October2023.csv', encoding='ISO-8859-1')
    print("File read successfully!")
    
    print(df['REFUSAL_DATE'].head())

    # Convert to datetime format
    df['REFUSAL_DATE'] = pd.to_datetime(df['REFUSAL_DATE'], format='%d-%b-%y', errors='coerce')

    # Map abbr to month names
    df['REFUSAL_DATE'] = df['REFUSAL_DATE'].dt.strftime('%d-%B-%Y')

except UnicodeDecodeError as e:
    print("Error decoding the file. Please check the file encoding.")
    print(e)

    with open('REFUSAL_ENTRY_2014-October2023.csv', 'rb') as f:
        preview = f.read(1000).decode('ISO-8859-1')

    print("Preview of the file content:")
    print(preview)

File read successfully!
0    09-Oct-20
1    07-Apr-20
2    27-Aug-19
3    23-Oct-17
4    29-Jun-16
Name: REFUSAL_DATE, dtype: object


## Cleaning month names

In [31]:
# mapping of month abbreviations to month names
month_mapping = {
    'Jan': 'January',
    'Feb': 'February',
    'Mar': 'March',
    'Apr': 'April',
    'May': 'May',
    'Jun': 'June',
    'Jul': 'July',
    'Aug': 'August',
    'Sep': 'September',
    'Oct': 'October',
    'Nov': 'November',
    'Dec': 'December'
}

# Convert to required format and map
df['REFUSAL_DATE'] = pd.to_datetime(df['REFUSAL_DATE'], format='%d-%B-%Y', errors='coerce')

df['REFUSAL_DATE'] = df['REFUSAL_DATE'].dt.strftime('%d-%B-%Y')

## Task 1: Read the CSV file and create country_violations_2014-2023.csv

In [20]:

country_violations = df.groupby(['ISO_CNTRY_CODE', 'PROVINCE_STATE', 'CITY_NAME']).count().loc[:, ['ENTRY_NUM']]
country_violations.columns = ['VIOLATION_COUNT']

country_violations.to_csv('country_violations_2014-2023.csv')

## Task 2: Create country_violations_year_month_2014-2023.csv

In [32]:

# Extract and Group columns
df['YEAR'] = pd.to_datetime(df['REFUSAL_DATE'], format='%d-%B-%Y', errors='coerce').dt.year
df['MONTH'] = pd.to_datetime(df['REFUSAL_DATE'], format='%d-%B-%Y', errors='coerce').dt.strftime('%B')

country_violations_year_month = df.groupby(['YEAR', 'MONTH', 'ISO_CNTRY_CODE', 'PROVINCE_STATE', 'CITY_NAME']).count().loc[:, ['ENTRY_NUM']]
country_violations_year_month.columns = ['VIOLATION_COUNT']

country_violations_year_month.to_csv('country_violations_year_month_2014-2023.csv')

## Task 3: Find the city, country, province with the most violations in a single month

In [34]:
# Find row with the maximum violation count
max_violation_row = country_violations_year_month.loc[country_violations_year_month['VIOLATION_COUNT'].idxmax()]

print(max_violation_row)

# Extract information directly from the index
most_violated_city = max_violation_row.name[4]
most_violated_country = max_violation_row.name[2]
most_violated_province = max_violation_row.name[3]
violations_count = max_violation_row['VIOLATION_COUNT']
year = max_violation_row.name[0]
month = max_violation_row.name[1]

# Display the result
print(f"The city, country, and province with the most violations in a single month: {most_violated_city}, {most_violated_country}, {most_violated_province}")
print(f"Month and year: {month}/{year}, Violations count: {violations_count}")

VIOLATION_COUNT    613
Name: (2022, April, MX, Ciudad de Mexico, Ciudad De Mexico), dtype: int64
The city, country, and province with the most violations in a single month: Ciudad De Mexico, MX, Ciudad de Mexico
Month and year: April/2022, Violations count: 613


## Task 4: Find the 10 most frequent products in the IRR for 2018

In [35]:
# Filter data for the year 2018
df_2018 = df[df['YEAR'] == 2018]

# ascertain the column exists and perform logic
if 'PRDCT_CODE_DESC_TEXT' in df_2018.columns:
    top_products_2018 = df_2018['PRDCT_CODE_DESC_TEXT'].value_counts().head(10)

    # Display the result
    print("The 10 most frequent products in the IRR for 2018:")
    print(top_products_2018)
else:
    print("Column 'PRDCT_CODE_DESC_TEXT' not found in the DataFrame 'df_2018'.")


The 10 most frequent products in the IRR for 2018:
PRDCT_CODE_DESC_TEXT
SUNGLASSES (NON-PRESCRIPTION INCLUDING PHOTOSENSITIVE)                                               575
MISCELLANEOUS PATENT MEDICINES, ETC.                                                                 433
VITAMIN, MINERAL, PROTEINS AND UNCONVENTIONAL DIETARY SPECIALITIES FOR HUMANS AND ANIMALS, N.E.C.    298
TUNA (ALBACORE, YELLOWFIN, BLUEFIN, SKIPJACK, ETC.)                                                  244
HERBALS & BOTANICALS (NOT TEAS), N.E.C.                                                              237
MAHI MAHI                                                                                            190
ULTRAVIOLET SCREEN/SUNSCREEN N.E.C.                                                                  183
GLOVE, PATIENT EXAMINATION, POLY                                                                     173
BATH SOAPS AND DETERGENTS (NOT ANTIPERSPIRANT) (PERSONAL CLEANLINESS)                   

## Task 5: Find the company associated with the largest violation in a single month

In [ ]:
# # Find the row with the maximum violation count in a single month
# max_violation_row_monthly_index = country_violations_year_month.groupby(['YEAR', 'MONTH'])['VIOLATION_COUNT'].idxmax()
# max_violation_row_monthly = country_violations_year_month.loc[max_violation_row_monthly_index]

# # Extract the corresponding indices for the largest violation
# indices = [index for index_tuple in max_violation_row_monthly_index for index in index_tuple]
# largest_violation_entry_num = df.loc[indices, 'ENTRY_NUM'].values[0]

# # Find the company associated with the largest violation
# # Flatten the MultiIndex and use it in loc
# largest_violation_entry_num_index = tuple(indices)
# largest_violation_entry_num = df.loc[largest_violation_entry_num_index, 'ENTRY_NUM'].values[0]

# # Find the company associated with the largest violation
# largest_violation_company = df.loc[df['ENTRY_NUM'] == largest_violation_entry_num, 'LGL_NAME'].values[0]

# print(f"The company associated with the largest violation in a single month: {largest_violation_company}")